# Ohio Scraping 

In [1]:
# set up 
import pandas as pd 
import requests 
import bs4
import numpy as np
import time 
import re
from tika import parser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from io import BytesIO
from urllib.request import urlopen

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

%xmode Minimal

Exception reporting mode: Minimal


In [2]:
# # set headers
# header_list = {'User-Agent': REDACTED,
#               'Accept-Language': 'en-US,en;q=0.9'}

In [82]:
# state - UPDATE EACH TIME
state = 'ohio'

In [ ]:
# review of robots.txt - no restrictions

In [ ]:
# test = pd.read_csv(f'{state}_2023.csv')
# test.head()

## Scrape links
`Selenium`

In [63]:
# get all links 
driver = webdriver.Chrome()
driver.get('https://governor.ohio.gov/media/news-and-media')

time.sleep(25)
#wait = WebDriverWait(driver, 15)

results = driver.find_elements(By.TAG_NAME, 'a')

links = [i.get_attribute('href') for i in results]
links = [i for i in links if "https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/" in i]

driver.close()

In [64]:
links

['https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/!ut/p/z1/04_Sj9CPykssy0xPLMnMz0vMAfIjo8zi_SzdHQ0NvQ383C0sDAwCnf2MTS3c3VwNAg30w_Eo8LV0NNePIka_AQ7gaEBIvxchBUAfGBX5Ovum60cVJJZk6GbmpeXrR-SllhfrJual6OampmQmAl0RhccciD_ACvA4tCA3NKLKJy3Y01FREQCN641y/#',
 'https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-lt-governor-husted-celebrate-historic-investments-in-new-operating-budget',
 'https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-lt-governor-husted-announce-career-tech-construction-grant-awards-new-equipment-funding-opportunity',
 'https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-lt-governor-husted-joby-aviation-announce-historic-manufacturing-site-in-ohio',
 'https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-launches-readohio-initiative',
 'https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-de

In [76]:
# hard code the index for the 2023 dates (previous methods to filter weren't working)
start = links.index('https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-vetoes-house-bill-68')
print('start index: ' + str(start))

end = links.index('https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/Governor-DeWine-Signs-Bills-Into-Law-01032023')
print('end index: ' + str(end))

start index: 53
end index: 312


In [78]:
# subset to 2023 links
links_2023 = links[53:313]
len(links_2023)

260

In [81]:
df_links_2023 = pd.DataFrame({'link': links_2023})
df_links_2023

,link
0,https://governor.ohio.gov/wps/portal/gov/gover...
1,https://governor.ohio.gov/wps/portal/gov/gover...
2,https://governor.ohio.gov/wps/portal/gov/gover...
3,https://governor.ohio.gov/wps/portal/gov/gover...
4,https://governor.ohio.gov/wps/portal/gov/gover...
...,...
255,https://governor.ohio.gov/wps/portal/gov/gover...
256,https://governor.ohio.gov/wps/portal/gov/gover...
257,https://governor.ohio.gov/wps/portal/gov/gover...
258,https://governor.ohio.gov/wps/portal/gov/gover...


In [83]:
df_links_2023.to_csv(f'{state}_links.csv', index = False)

In [ ]:
# add blank content, title, and date columns 
metadata = {'content': [np.nan], 
           'title': [np.nan], 
           'date': [np.nan]}

metadata_df = pd.DataFrame(metadata)

links_df = pd.concat([df_links_2023, metadata_df], axis = 1)

In [101]:
links_df.head()

,link,content,title,date
0,https://governor.ohio.gov/wps/portal/gov/gover...,NaN,NaN,NaN
1,https://governor.ohio.gov/wps/portal/gov/gover...,NaN,NaN,NaN
2,https://governor.ohio.gov/wps/portal/gov/gover...,NaN,NaN,NaN
3,https://governor.ohio.gov/wps/portal/gov/gover...,NaN,NaN,NaN
4,https://governor.ohio.gov/wps/portal/gov/gover...,NaN,NaN,NaN


## Scrape content 
`Selenium`

### Test on one link

In [98]:
# test one link  
driver = webdriver.Chrome()
driver.get("https://governor.ohio.gov/media/news-and-media/Governor-DeWine-Signs-Bills-Into-Law-01032023")
#time.sleep(5)

wait = WebDriverWait(driver, 5)

title = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'odx-content__title'))).text

date = driver.find_element(By.XPATH, '//*[@id="odx-main-content"]/article/div/div[1]/div/span').text

content = driver.find_element(By.XPATH,'//*[@id="js-odx-content__body"]').text

driver.quit()

'Governor DeWine Signs Bills Into Law'

In [108]:
# try on a small amount
for link in links_df.link[0:6]:
    get_ohio_content(link, links_df) 

'content already existing for https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-vetoes-house-bill-68'

'content already existing for https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-signs-bill-into-law-12-28'

'content already existing for https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-issues-statement-on-death-of-former-us-senator-herb-kohl'

'content already existing for https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-announces-financial-support-for-east-palestine-businesses'

'content already existing for https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-announces-state-support-of-46-hisotric-rehabilitation-projects'

'content already existing for https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/ohio-governors-expedited-pardon-project-surpasses-100-pardons-for-reformed-citizens'

In [ ]:
# create function
def get_ohio_content(link, df): 
    try: 
        # check if content has already been pulled
        result = df.loc[df.link == link, 'content'].notnull().all()
        
        if result:
            return f'content already existing for {link}' 
        else: 
            try: 
                driver = webdriver.Chrome()
                driver.get(link)
                wait = WebDriverWait(driver, 5)
                
                title = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'odx-content__title'))).text
                date = driver.find_element(By.XPATH, '//*[@id="odx-main-content"]/article/div/div[1]/div/span').text
                content = driver.find_element(By.XPATH,'//*[@id="js-odx-content__body"]').text
                
                driver.quit()
                df.loc[df.link == link, 'content'] = content
                df.loc[df.link == link, 'title'] = title 
                df.loc[df.link == link, 'date'] = date 
                return f"success: content added from {link}"
            except: 
                pass
    except Exception as e: 
        return f"unable to gather content from {link} due to {e}"

### Scrape content from all links 

In [109]:
for link in links_df.link:
    get_ohio_content(link, links_df) 

'content already existing for https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-vetoes-house-bill-68'

'content already existing for https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-signs-bill-into-law-12-28'

'content already existing for https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-issues-statement-on-death-of-former-us-senator-herb-kohl'

'content already existing for https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-announces-financial-support-for-east-palestine-businesses'

'content already existing for https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-announces-state-support-of-46-hisotric-rehabilitation-projects'

...

'success: content added from https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/Governor-DeWine-Announces-Staff-Appointments-01062023'

'success: content added from https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/Governor-DeWine-Signs-Bills-Into-Law-Vetoes-HB+513-01052023'

'success: content added from https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/Governor-DeWine-Signs-Bills-Into-Law-Vetoes-HB-286-01032023'

'success: content added from https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/Governor-DeWine-Signs-Bill-that-Strengthens-Distracted-Driving-Laws-in-Ohio-01032023'

'success: content added from https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/Governor-DeWine-Signs-Bills-Into-Law-01032023'

## Validate and Clean

In [111]:
links_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260 entries, 0 to 259
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   link     260 non-null    object
 1   content  258 non-null    object
 2   title    258 non-null    object
 3   date     258 non-null    object
dtypes: object(4)
memory usage: 10.2+ KB


In [116]:
# check null values
missing = links_df[links_df.content.isna()]
missing

,link,content,title,date
102,https://governor.ohio.gov/wps/portal/gov/gover...,NaN,NaN,NaN
163,https://governor.ohio.gov/wps/portal/gov/gover...,NaN,NaN,NaN


In [117]:
# try again 
for link in missing.link:
    get_ohio_content(link, links_df) 

'success: content added from https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/governor-dewine-lt-governor-husted-announce-new-projects-expected-to-create-more-than-900-jobs-58-million-in-payroll'

'success: content added from https://governor.ohio.gov/wps/portal/gov/governor/media/news-and-media/052523'

In [118]:
links_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260 entries, 0 to 259
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   link     260 non-null    object
 1   content  260 non-null    object
 2   title    260 non-null    object
 3   date     260 non-null    object
dtypes: object(4)
memory usage: 10.2+ KB


In [119]:
print(f"pulled {str(len(links_df))} documents from {state} for 2023")

pulled 260 documents from ohio for 2023


In [122]:
# convert to date-time format
links_df['date_clean'] = pd.to_datetime(links_df['date'])

In [123]:
links_cl = links_df.drop(columns=['date']).copy()

In [125]:
links_cl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260 entries, 0 to 259
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   link        260 non-null    object        
 1   content     260 non-null    object        
 2   title       260 non-null    object        
 3   date_clean  260 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 10.2+ KB


## Export

In [126]:
# export
links_cl.to_csv(f'{state}_2023.csv', index = False)

In [127]:
test = pd.read_csv(f'{state}_2023.csv')
test.head()

,link,content,title,date_clean
0,https://governor.ohio.gov/wps/portal/gov/gover...,"(COLUMBUS, Ohio)— Ohio Governor Mike DeWine to...",Governor DeWine Vetoes House Bill 68,2023-12-29
1,https://governor.ohio.gov/wps/portal/gov/gover...,"(COLUMBUS, Ohio)—Ohio Governor Mike DeWine sig...",Governor DeWine Signs Bills Into Law,2023-12-28
2,https://governor.ohio.gov/wps/portal/gov/gover...,"(COLUMBUS, Ohio)—Ohio Governor Mike DeWine tod...",Governor DeWine Issues Statement on Death of F...,2023-12-28
3,https://governor.ohio.gov/wps/portal/gov/gover...,"(EAST PALESTINE, Ohio)— Ohio Governor Mike DeW...",Governor DeWine Announces Financial Support fo...,2023-12-26
4,https://governor.ohio.gov/wps/portal/gov/gover...,"(COLUMBUS, Ohio)— Ohio Governor Mike DeWine, L...",Governor DeWine Announces State Support of 46 ...,2023-12-21
